## Imports

In [17]:
import time
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup

## Driver

In [18]:
driver = webdriver.Chrome(executable_path="chromedriver")

## Web Scraping - Twitter Search

In [19]:
def get_tweets():
    tweets = []

    page_source = driver.page_source
    soup = BeautifulSoup(page_source, "lxml")

    articles = soup.find_all("article")

    for article in articles:
        div = article.find_next("div", {"lang": "en"})
        if div is not None:
            tweets.append(div.getText())

    return tweets

In [20]:
def scrape_results(url):
    all_tweets = []

    driver.get(url)
    time.sleep(10)
    all_tweets = all_tweets + get_tweets()

    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
        all_tweets = all_tweets + get_tweets()

        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    return all_tweets

## Unemployment Tweets

In [21]:
unemployment_query = "https://twitter.com/search?q=%23malaysia%20%23unemployment%20since%3A2020-03-18%20lang%3Aen&src=typed_query&f=live"
unemployment_tweets = scrape_results(unemployment_query)
unemployment_tweets

['#Unemployment below 700,000, lowest since April 2020 \n\nhttps://themalaysianreserve.com/2022/01/10/unemployment-below-700000-lowest-since-april-2020/…\n#Malaysia #TMRNews #TMRPhoto',
 'Throwback to this opinion piece I wrote on June 13 last year about the worry among young adults over job security and career prospects amid the pandemic.\n\n#Malaysia #unemployment\nhttps://thestar.com.my/metro/views/2020/06/13/youths-anxious-about-the-future…',
 'Malaysia Witnesses Influx Of Indonesian Migrants #migrants #indonesia #m... https://youtu.be/83Ps-DVbJQ0 via @YouTube #migrants #indonesia #malaysia #unemployment #asia #sea #laborers #wages',
 'Unemployment levels over 30 years in Malaysia has been below 5% consistently #malaysianeconomy #malaysia #MCO #unemployment',
 'Happy to see unemployment levels dipping in Malaysia since MCO has been lifted. #malaysia #economy #bursa #klci #jobs #unemployment',
 'Facebook livestreaming: @myWISDOMF @Madiustangau\nThank you and see you there! \n#WISDOM

In [22]:
df_unemployment = pd.DataFrame({"tweet": unemployment_tweets})
df_unemployment.to_csv("../data/unemployment_tweets.csv", index=False)